### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import time

### Importing the Dataset

In [2]:
env = pd.read_csv('Ads_CTR_Optimisation.csv')

In [3]:
env.head()

,Ad 1,Ad 2,Ad 3,Ad 4,Ad 5,Ad 6,Ad 7,Ad 8,Ad 9,Ad 10
0,1,0,0,0,1,0,0,0,1,0
1,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0


### Random Policy
If we were to not have Reinforcement Learning, we would place the ads randomly at given positions. We will now simulate the same.

In [4]:
reward = 0
for x in range(len(env)):
    action = np.random.randint(0,10)
    if env.values[x][action]==1:
        reward+=1
print('rewards:', reward)

rewards: 1229


### Using Max Policy
Another question we might ask: is it right to display the ad where it is clicked the most number of times. For instance, there might be a certain position where the ad clicked with a higher probability. Since the values of the rows is either 1 or 0, we can sum across the columns and count the number of times ad in the position was clicked.

In [5]:
clicked_counts = env.values.sum(axis=0)
counts = pd.DataFrame({'ad': np.arange(1,11), 'count': clicked_counts})
counts.set_index('ad')

,count
ad,
1,1703
2,1295
3,728
4,1196
5,2695
6,126
7,1112
8,2091
9,952


Which indicates Ad 5 was clicked 2695. So if we were to always place ad on position 5, it would be click around 2695 times. But can we do better?

### Dynamic Programming (Policy Iteration)

In [6]:
len(env)

10000

In [7]:
state_size=len(env)
state_list=[]
for state in range(state_size):
    state_list.append(state)

In [8]:
start = time.time()
# action could be placing the ad on any of the 10 positions
action_space = np.arange(0, 10)

policy = [random.choice(action_space) for x in range(state_size)]
# will take random action for the first time
first_time = True

# delta
small_change = 1e-20
# discount factor
gamma = 0.9
episodes = 0
max_episodes = 100

V = dict()
# last positions reward will be 1 - terminal state
V[10000] = 1

# initially the value function for all states will be random values close to zero
for i in range(state_size):
    V[i] = np.random.random()
deltas = []
while episodes < max_episodes:
    # policy evaluation (until convergence of state value function)
    while True:
        if episodes > max_episodes:
            break
        episodes += 1
        if episodes % 100 == 0:
            print("Current episode: {}".format(episodes))
        biggest_change = 0
        # loop through every state present
        for state in range(state_size):
            old_V = V[state]
            # take random action according to policy
            action = policy[state]
            #print(action)
            new_state = random.choice(list(set(state_list) - set([state])))
            #print(new_state)
            reward = env.values[state][action]
            #
            V[state] = (reward + gamma * V[new_state])/9999
            # We're calculating biggest change to have an idea on convergence. 
            # Initially, the changes will be huge, but as 
            # we update the values, they will tend towards a convergence point
            biggest_change = max(biggest_change, abs(V[state] - old_V))
        deltas.append(biggest_change)
        if biggest_change < small_change:
            break
            
    # policy improvement
    policy_changed = False
    for state in range(state_size):
        best_val = -np.inf
        best_action = -1
        for action in action_space:
            new_state = random.choice(list(set(state_list) - set([state])))
            reward = env.values[state][action]
            # calculate the action with the best
            # future reward
            future_reward = (reward + gamma * V[new_state])/9999
            if future_reward > best_val:
                best_val = future_reward
                best_action = action
        assert best_action != -1
        # After convergence, the policy will not change since we would have already reached
        # the optimum policy. So check if the policy is not updated, if not then stop. 
        if policy[state] != best_action:
            policy_changed = True
        policy[state] = best_action

    if not policy_changed:
        break

end = time.time()
print("Time taken in seconds: ", end-start)
print("Total episodes trained: {}".format(episodes))

KeyboardInterrupt: 

In [ ]:
plt.rcParams["figure.figsize"] = (8,4)
plt.plot(deltas, label="deltas")
plt.legend()
plt.title('Convergence plot: Policy iteration')
plt.tight_layout()

In [ ]:
reward = 0
for x in range(len(env)):
    action = policy[x]
    if env.values[x][action]==1:
        reward+=1
print('rewards:', reward)
    